In [47]:
# Cell 1: Imports and Setup

import os
import random
from datetime import datetime
from faker import Faker
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image, KeepTogether
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch

# Set up directories
BASE_DIR = r"C:\Users\cha\Desktop\a_pdf"
LOGO_PATH = os.path.join(BASE_DIR, "Sample_Logos", "chase_bank_logo.png")
OUTPUT_DIR = os.path.join(BASE_DIR, "out")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Initialize Faker for synthetic data
faker = Faker()


In [62]:

# Cell 2: Generate synthetic data
def generate_statement_data():
    last_four = faker.random_number(digits=4)
    account_type = faker.random_element(elements=("Chase Total Checking", "Chase Complete Checking"))
    deposits = [
        {
            "date": faker.date_this_month().strftime("%m/%d/%Y"),
            "description": faker.random_element(elements=("Salary Deposit", "Online Transfer", "Cash Deposit")),
            "amount": round(faker.random_number(digits=4) / 100, 2)
        } for _ in range(faker.random_int(1, 3))
    ]
    withdrawals = [
        {
            "date": faker.date_this_month().strftime("%m/%d/%Y"),
            "description": faker.random_element(elements=("Grocery Shopping", "Utility Payment", "Online Purchase")),
            "amount": -round(faker.random_number(digits=4) / 100, 2)
        } for _ in range(faker.random_int(1, 3))
    ]
    daily_balances = [
        {
            "date": faker.date_this_month().strftime("%m/%d/%Y"),
            "description": faker.random_element(elements=("Daily Balance", "End of Day Balance")),
            "amount": round(faker.random_number(digits=5) / 100, 2)
        } for _ in range(faker.random_int(2, 5))
    ]
    beginning_balance = round(faker.random_number(digits=5) / 100, 2)
    return {
        "account_holder": faker.name(),
        "account_holder_address": faker.address().replace("\n", ", "),
        "account_number": f"****{last_four}",
        "account_number_filename": f"{last_four}",
        "account_type": account_type,
        "statement_date": datetime.now().strftime("%B %d, %Y"),
        "statement_period": f"{(datetime.now().replace(day=1)).strftime('%B %d, %Y')} - {datetime.now().strftime('%B %d, %Y')}",
        "summary": {
            "beginning_balance": f"${beginning_balance:,.2f}",
            "deposits_count": len(deposits),
            "deposits_total": f"${sum(t['amount'] for t in deposits):,.2f}",
            "withdrawals_count": len(withdrawals),
            "withdrawals_total": f"${abs(sum(t['amount'] for t in withdrawals)):,.2f}",
            "transactions_count": len(deposits) + len(withdrawals),
            "ending_balance": f"${beginning_balance + sum(t['amount'] for t in deposits) + sum(t['amount'] for t in withdrawals):,.2f}"
        },
        "deposits": deposits,
        "withdrawals": withdrawals,
        "daily_balances": daily_balances,
        "show_fee_waiver": faker.random_element(elements=(True, False))
    }


In [63]:
def create_card_content(content, width):
    # Wrap content in a Table to simulate card with border and background
    card_table = Table([[content]], colWidths=[width - 20])
    card_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, -1), colors.white),
        ('BOX', (0, 0), (-1, -1), 1, colors.HexColor("#e5e7eb")),
        ('INNERGRID', (0, 0), (-1, -1), 0, colors.transparent),
        ('VALIGN', (0, 0), (-1, -1), 'TOP'),
        ('LEFTPADDING', (0, 0), (-1, -1), 10),
        ('RIGHTPADDING', (0, 0), (-1, -1), 10),
        ('TOPPADDING', (0, 0), (-1, -1), 10),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 10),
    ]))
    return KeepTogether(card_table)

# Define styles
def get_styles():
    styles = getSampleStyleSheet()
    return {
        "title": ParagraphStyle(name='Title', fontSize=18, textColor=colors.HexColor("#003087"), fontName="Helvetica-Bold", spaceAfter=10, leading=20),
        "normal": ParagraphStyle(name='Normal', fontSize=10, textColor=colors.HexColor("#1f2937"), fontName="Helvetica", spaceAfter=5),
        "bold": ParagraphStyle(name='Bold', fontSize=10, textColor=colors.HexColor("#1f2937"), fontName="Helvetica-Bold", spaceAfter=5),
        "blue_bold": ParagraphStyle(name='BlueBold', fontSize=10, textColor=colors.HexColor("#003087"), fontName="Helvetica-Bold", spaceAfter=5),
        "customer_service": ParagraphStyle(name='CustomerService', fontSize=10, textColor=colors.white, fontName="Helvetica-Bold", backColor=colors.HexColor("#003087"), spaceAfter=5, borderPadding=5, borderColor=colors.HexColor("#003087"), borderWidth=1, borderRadius=4),
        "green_text": ParagraphStyle(name='GreenText', fontSize=10, textColor=colors.HexColor("#10b981"), fontName="Helvetica-Bold", spaceAfter=5),
        "red_text": ParagraphStyle(name='RedText', fontSize=10, textColor=colors.HexColor("#ef4444"), fontName="Helvetica-Bold", spaceAfter=5),
        "small": ParagraphStyle(name='Small', fontSize=10, textColor=colors.HexColor("#1f2937"), fontName="Helvetica", spaceAfter=5, leading=14.5),
        "small_summary": ParagraphStyle(name='SmallSummary', fontSize=12, fontName="Helvetica", spaceBefore=10)
    }

# Header layouts
def header_detailed(data, styles):
    content = []
    if os.path.exists(LOGO_PATH):
        img = Image(LOGO_PATH)
        if img._width and img._height:  # Check if dimensions are available
            img.drawHeight = img._height * inch / 72  # Convert pixels to inches based on 72 DPI
            img.drawWidth = img._width * inch / 72   # Maintain aspect ratio
        else:
            img.drawHeight = 0.5 * inch  # Default height if dimensions unavailable
            img.drawWidth = 1.5 * inch   # Default width to maintain reasonable size
        content.append(img)
    content.extend([
        Spacer(1, 0.1*inch),
        Paragraph("JPMorgan Chase Bank, N.A.", styles["bold"]),
        Paragraph("PO Box 659754, San Antonio, TX 78265-9754", styles["normal"]),
        Paragraph(f"Statement for: {data['account_holder']}", styles["normal"]),
    ])
    right_content = [
        Paragraph(f"Period: {data['statement_period']}", styles["blue_bold"]),  # Moved here from left content
        Paragraph(f"Account Number: {data['account_number']}", styles["normal"]),
        Spacer(1, 0.1*inch),
        Paragraph("Customer Service", styles["customer_service"]),
        Paragraph("chase.com | 1-800-242-7338", styles["normal"]),
        Paragraph("Español: 1-888-622-4273 | International: 1-713-262-1679", styles["normal"])
    ]
    # Use a single row table with dynamic height
    table = Table([[content, right_content]], colWidths=[3*inch, 3*inch])
    table.setStyle(TableStyle([
        ('VALIGN', (0, 0), (-1, -1), 'TOP'),
        ('ALIGN', (1, 0), (1, -1), 'RIGHT'),  # Ensure right column aligns right
        ('BACKGROUND', (0, 0), (-1, -1), colors.white),
        ('LEFTPADDING', (0, 0), (-1, -1), 0),  # Remove extra padding to fit content
        ('RIGHTPADDING', (0, 0), (-1, -1), 0),
        ('TOPPADDING', (0, 0), (-1, -1), 0),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 0),
    ]))
    # Wrap the table in create_card_content for card styling
    return create_card_content([table], 7*inch)

In [64]:

# Cell 4: Create PDF
def create_bank_statement(data, layout_config=None):
    output_file = os.path.join(OUTPUT_DIR, f"chase_statement_{data['account_number_filename']}.pdf")
    doc = SimpleDocTemplate(output_file, pagesize=letter, topMargin=0.5*inch, bottomMargin=0.5*inch, leftMargin=0.5*inch, rightMargin=0.5*inch)
    styles = get_styles()
    elements = []

    # Default layout configuration
    if layout_config is None:
        layout_config = {
            "header_type": random.choice(["standard", "compact", "detailed"]),
            "sections": ["header", "payee_info", "checking_summary", "important_account_info", "transactions", "daily_balance", "footnotes"]
        }

    section_functions = {
        "header": {
            "detailed": lambda d, s: header_detailed(d, s)
        },
        "payee_info": payee_info,
        "checking_summary": checking_summary,
        "important_account_info": important_account_info,
        "transactions": transactions,
        "daily_balance": daily_balance,
        "footnotes": footnotes
    }

    # Build sections based on layout_config
    for section in layout_config["sections"]:
        if section == "header":
            header_func = section_functions["header"][layout_config["header_type"]]
            elements.append(header_func(data, styles))
        else:
            elements.append(section_functions[section](data, styles))
        elements.append(Spacer(1, 0.2*inch))

    # Build PDF
    doc.build(elements)
    print(f"PDF generated at: {output_file}")


In [65]:
if __name__ == "__main__":
    statement_data = generate_statement_data()
    # Example of custom layout configuration
    layout_config = {
        "header_type": "detailed",  # Changed to fixed "detailed" to match available function
        "sections": ["header", "payee_info", "checking_summary", "important_account_info", "transactions"]
    }
    create_bank_statement(statement_data, layout_config)

PDF generated at: C:\Users\cha\Desktop\a_pdf\out\chase_statement_4605.pdf
